In [2]:
# https://python.langchain.com/en/latest/modules/agents/getting_started.html 
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate, 
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
    
)
from langchain.schema import ( 
    AIMessage,
    HumanMessage,
    SystemMessage,
)

from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# from langchain.llms import OpenAI

In [3]:
import openai
import os
import dotenv
dotenv.load_dotenv()


# get above env variables using os.environ.get and use provided values as defaults
gpt4_endpoint = os.environ.get('gpt4_endpoint', 'https://lnc-eastus-openai.openai.azure.com/')
gpt4_key = os.environ.get('gpt4_key', '121221')
gpt4_region = os.environ.get('gpt4_region', 'eastus')
gpt4_modelid_4k = os.environ.get('gpt4_modelid_4k', 'gpt-4-0314')
gpt4_modelid_32k = os.environ.get('gpt4_modelid_32k', 'gpt-4-32k-0314')
gpt4_api_version = os.environ.get('gpt4_api_version', '2023-03-15-preview')


# get the above from os.environ.get and use provided values as defaults
chatgpt_api_key = os.environ.get('chatgpt_api_key', '121212121212')
chatgpt_region = os.environ.get('chatgpt_region', 'southcentralus')
chatgpt_endpoint = os.environ.get('chatgpt_endpoint', 'https://magopenai.openai.azure.com/')
chatgpt_api_version = os.environ.get('chatgpt_api_version', '2021-03-15-preview')

oai_api_key = os.environ.get('oai_api_key', '121212121212')

print(f'gpt4_endpoint: {gpt4_endpoint}')
print(f'gpt4_key: **********')
print(f'gpt4_region: {gpt4_region}')
print(f'gpt4_modelid_4k: {gpt4_modelid_4k}')
print(f'gpt4_modelid_32k: {gpt4_modelid_32k}')
print(f'gpt4_api_version: {gpt4_api_version}')
print(f'chatgpt_api_key: **********')
print(f'chatgpt_region: {chatgpt_region}')
print(f'chatgpt_endpoint: {chatgpt_endpoint}')
print(f'oai_api_key: **********')


gpt4_endpoint: https://lnc-eastus-openai.openai.azure.com/
gpt4_key: **********
gpt4_region: eastus
gpt4_modelid_4k: gpt-4-0314
gpt4_modelid_32k: gpt-4-32k-0314
gpt4_api_version: 2023-03-15-preview
chatgpt_api_key: **********
chatgpt_region: southcentralus
chatgpt_endpoint: https://magopenai.openai.azure.com/
oai_api_key: **********


In [4]:
# must set openai_api_base correctly -- but since thats done in the next call, AzureOpenAI instance will work
azure_llm = AzureOpenAI(
    deployment_name='text-davinci-003', 
    model_name='text-davinci-003', 
    openai_api_key=chatgpt_api_key,
        model_kwargs={
        "api_base": chatgpt_endpoint, 
        "api_type": "azure", 
        "api_version": chatgpt_api_version,
    },

    verbose=True)


In [6]:
azure_chat_model = AzureChatOpenAI(
    deployment_name=gpt4_modelid_4k,
    model_name=gpt4_modelid_4k,
    openai_api_key=gpt4_key,
    openai_api_base=gpt4_endpoint,
    openai_api_version=gpt4_api_version,
    verbose=True, 
    streaming=True,
    temperature=0)

In [9]:
from pprint import pprint
from langchain.llms import BaseLLM
pprint(azure_llm._identifying_params)
#cast azure_llm to BaseLLM
print(isinstance(azure_llm, BaseLLM))
# access parent class
print(azure_llm.__class__.__base__)
pprint(azure_chat_model._identifying_params)

{'api_base': 'https://magopenai.openai.azure.com/',
 'api_type': 'azure',
 'api_version': '2023-03-15-preview',
 'best_of': 1,
 'deployment_name': 'text-davinci-003',
 'frequency_penalty': 0,
 'logit_bias': {},
 'max_tokens': 256,
 'model_name': 'text-davinci-003',
 'n': 1,
 'presence_penalty': 0,
 'request_timeout': None,
 'temperature': 0.7,
 'top_p': 1}
True
<class 'langchain.llms.openai.BaseOpenAI'>
{'engine': 'gpt-4-0314',
 'max_tokens': None,
 'model': 'gpt-4-0314',
 'model_name': 'gpt-4-0314',
 'n': 1,
 'request_timeout': 60,
 'stream': True,
 'temperature': 0.0}


In [5]:
prompt = "Hello, I am a chatbot. How are you today?"
azure_llm(prompt)

"\n\nI'm doing well, thank you! How about you?"

In [8]:
azure_chat_model([HumanMessage(content=prompt)])

AIMessage(content="Hello! As an AI, I don't have feelings or emotions, but I'm here to help you with any questions or concerns you may have. How can I assist you today?", additional_kwargs={})

In [5]:
openai.api_key = gpt4_key
openai.api_base = gpt4_endpoint

openai.api_type = "azure"
openai.api_version = gpt4_api_version

# iterate over openai.Deployment
for d in openai.Deployment.list()['data']: 
    print(f'id: {d["id"]}, model: {d["model"]}')



id: text-davinci-003, model: text-davinci-003
id: gpt-35-turbo_v0301, model: gpt-35-turbo
id: gpt-4-0314, model: gpt-4
id: gpt-4-32k-0314, model: gpt-4-32k
id: text-embedding-ada-002, model: text-embedding-ada-002


In [6]:
# test if chat_model is an instance of AzureChatOpenAI
from langchain.llms import BaseLLM, GPT4All
print(f"azure_llm (AzureOpenAI): {isinstance(azure_llm, AzureOpenAI)}")
# isinstance(azure_llm, BaseLLM)
print(f"azure_llm (BaseLLM): {isinstance(azure_llm, BaseLLM)}")
print(f"azure_chat_model (AzureOpenAI): {isinstance(azure_chat_model, AzureOpenAI)}")
print(f"azure_chat_model (BaseLLM): {isinstance(azure_chat_model, BaseLLM)}")
print(f"azure_chat_model (AzureChatOpenAI): {isinstance(azure_chat_model, AzureChatOpenAI)}")
isinstance(GPT4All, BaseLLM)

# does GPT4All extend BaseLLM?
issubclass(GPT4All, BaseLLM)
issubclass(AzureOpenAI, BaseLLM)
issubclass(AzureChatOpenAI, BaseLLM)

azure_llm (AzureOpenAI): True
azure_llm (BaseLLM): True
azure_chat_model (AzureOpenAI): False
azure_chat_model (BaseLLM): False
azure_chat_model (AzureChatOpenAI): True


False

In [19]:
import langchain
# import HumanMessage


print(dir(langchain.chat_models.AzureChatOpenAI))
print(AzureChatOpenAI.Config.extra)

azure_chat_model.call_as_llm("Hello, my name is John. What is your name?")


['Config', '__abstractmethods__', '__annotations__', '__call__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_agenerate', '_calculate_keys', '_combine_llm_outputs', '_copy_and_set_values', '_create_chat_result', '_create_m

In [60]:
azure_chat_model.streaming = False
resp = azure_chat_model(
    [HumanMessage(content="Write me a kids scifi story, starts with 'Once upon a time...'")],
    stop=["\n", "Human:", "AI:"],
    )

if azure_chat_model.streaming is not True: 
    print(resp.content)


Once upon a time, in a small village on the planet Zog, there lived a young boy named Zippy. Zippy was not like the other children in the village. He had a wild imagination and a thirst for adventure. He would often daydream about exploring the universe and discovering new planets.


In [59]:
# print(azure_chat_model.streaming)
# print(azure_chat_model.callback_manager)

azure_chat_model.streaming = True
azure_chat_model.callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])

azure_chat_model(
    [HumanMessage(content="Write me a kids scifi story, starts with 'Once upon a time...'")],
    # stop=["\n", "Human:", "AI:"],
    )

print('The End!')

Once upon a time, in a small village on the planet Zog, there lived a young boy named Zippy. Zippy was not like the other children in the village. He had a wild imagination and a thirst for adventure. He would often daydream about exploring the universe and discovering new planets.

One day, while Zippy was playing in the fields, he stumbled upon a strange-looking rock. It was unlike any rock he had ever seen before. It was smooth and shiny, and it seemed to glow with a mysterious energy. Zippy picked up the rock and felt a sudden surge of power coursing through his body.

As he held the rock, Zippy noticed that it began to change shape. It transformed into a small, metallic device with buttons and dials. Zippy couldn't believe his eyes. He had found a real-life spaceship!

Without hesitation, Zippy climbed into the tiny spaceship and began to explore its controls. He pressed a button, and the spaceship's engines roared to life. The spaceship lifted off the ground and soared into the s

In [62]:
dir(langchain.llms)

['AI21',
 'AlephAlpha',
 'Anthropic',
 'AzureOpenAI',
 'Banana',
 'BaseLLM',
 'CerebriumAI',
 'Cohere',
 'DeepInfra',
 'Dict',
 'ForefrontAI',
 'GPT4All',
 'GooseAI',
 'HuggingFaceEndpoint',
 'HuggingFaceHub',
 'HuggingFacePipeline',
 'LlamaCpp',
 'Modal',
 'NLPCloud',
 'OpenAI',
 'OpenAIChat',
 'Petals',
 'PromptLayerOpenAI',
 'PromptLayerOpenAIChat',
 'RWKV',
 'Replicate',
 'SagemakerEndpoint',
 'SelfHostedHuggingFaceLLM',
 'SelfHostedPipeline',
 'StochasticAI',
 'Type',
 'Writer',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'ai21',
 'aleph_alpha',
 'anthropic',
 'bananadev',
 'base',
 'cerebriumai',
 'cohere',
 'deepinfra',
 'forefrontai',
 'gooseai',
 'gpt4all',
 'huggingface_endpoint',
 'huggingface_hub',
 'huggingface_pipeline',
 'llamacpp',
 'loading',
 'modal',
 'nlpcloud',
 'openai',
 'petals',
 'promptlayer_openai',
 'replicate',
 'rwkv',
 'sagemaker_endpoint',
